In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
gis = arcgis.GIS()

from IPython.display import display, Markdown

In [2]:
#get root folder
dir_working = os.getcwd()
print(dir_working)

#define folders
dir_data     = os.path.join(dir_working, "data"        )
dir_process  = os.path.join(dir_working, "intermediate")
dir_results  = os.path.join(dir_working, "results"     )
dir_inputs   = os.path.join(dir_working, "input"       )
dir_webapp   = os.path.join(dir_working, "forWebApp"   )

ClassParameters    = os.path.join(dir_inputs, r"class_parameters.csv"     )

ProcessGDB = "process.gdb"
WebAppGDB  = "centerstod_webapp.gdb"
ParcelsDevCategory = os.path.join(dir_process, ProcessGDB + "\ParcelsAA_DevCategory")
ParcelsWebApp      = os.path.join(dir_webapp , WebAppGDB  + "\AAParcels"            )
DevCatWebApp       = os.path.join(dir_webapp , WebAppGDB  + "\AADevCategories"      )

#name of new data features
AnalysisAreas      = os.path.join(dir_process, "analysisareas.shp"        )
AnalysisAreas_IDs  = os.path.join(dir_process, "analysisareas_ids.shp"    )
ParcelsAA          = os.path.join(dir_process, "parcelsaa.shp"            )
AnalysisAreasTable = os.path.join(dir_results, "areas.csv"                )

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)


E:\GitHub\Residential-Capacity


In [3]:
AnalysisAreas

'E:\\GitHub\\Residential-Capacity\\intermediate\\analysisareas.shp'

In [4]:
sdf_AA = pd.DataFrame.spatial.from_featureclass(AnalysisAreas)
sdf_AA

,FID,city,county,city_lu_ty,gen_lu_typ,max_dua,code,plan_year,plan_sourc,data_sourc,Shape_Leng,Shape_Area,AnalysisAr,Analysis_1,SHAPE
0,0,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,599.400903,12203.821403,Bluffdale,a4,"{""rings"": [[[420271.3705000002, 4480517.338500..."
1,1,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,5673.401678,395913.547830,Bluffdale,a5,"{""rings"": [[[421136.0301000001, 4481104.8302],..."
2,2,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,1554.841774,85616.693206,Bluffdale,a4,"{""rings"": [[[420636.9475999996, 4481132.772299..."
3,3,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a4,2019.0,,,254.967270,3811.045695,Bluffdale,a4,"{""rings"": [[[420671.3481999999, 4481261.932600..."
4,4,Bluffdale,Salt Lake,Cluster Residential,Residential SF,1.0,a5,2019.0,,,2588.398617,261893.287379,Bluffdale,a5,"{""rings"": [[[421931.24289999995, 4482241.10689..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11136,11136,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,101.746609,647.000097,Payson,g1,"{""rings"": [[[437436.2742999997, 4432295.444599..."
11137,11137,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,347.071602,6455.345939,Payson,g1,"{""rings"": [[[437517.15270000044, 4432410.1381]..."
11138,11138,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,441.004580,8873.413053,Payson,g1,"{""rings"": [[[437390.2474999996, 4432411.0495],..."
11139,11139,Payson,Utah,Mixed Use Neighborhood,Mixed Use,12.0,g1,2020.0,https://paysonutah.org/storage/2016/05/General...,https://paysonutah.org/storage/2016/05/General...,743.393043,29068.043955,Payson,g1,"{""rings"": [[[439725.0575000001, 4433219.461200..."


# Parcels

In [5]:
sdf_ParcelsAA = pd.DataFrame.spatial.from_featureclass(ParcelsAA)
sdf_ParcelsAA.groupby(['AreaID','ClassID'], as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum]})

,AreaID,ClassID,parcel_id,job_spaces,resunits,Acres
,,,size,sum,sum,sum
0,0,a1,575.0,7.206084,93.060474,1554.873664
1,0,a2,4642.0,232.118444,1262.263010,7983.559693
2,0,a3,4597.0,292.415124,448.658337,11627.908363
3,0,a4,617.0,120.233696,18.102295,905.944246
4,0,a5,1203.0,267.243745,279.445881,2452.467821
...,...,...,...,...,...,...
647,76,a3,34.0,0.000000,0.000957,10.513478
648,77,d1,2842.0,108.960528,2521.094418,666.306789
649,77,d2,85.0,24.961343,67.445078,13.757016


In [6]:
sdf_ParcelsAA['county_id']

0         4.0
1         4.0
2         4.0
3         4.0
4         4.0
         ... 
939712    1.0
939713    1.0
939714    1.0
939715    1.0
939716    1.0
Name: county_id, Length: 939717, dtype: float64

# Land Use

In [7]:
display(sdf_ParcelsAA.columns)
sdf_ParcelsAA = sdf_ParcelsAA.rename(columns={"luType": "luTypeEx"})
display(sdf_ParcelsAA.columns)

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height', 'luType',
       'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt', 'resunits',
       'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re', 'sfvalue_co',
       'value_res', 'value_com', 'ParcelAcre', 'PieceAcres', 'PiecePorti',
       'Acres', 'SHAPE'],
      dtype='object')

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [8]:
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Residential'), 'luTypeEx'] = "Residential"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & (sdf_ParcelsAA['bldgtype'] == 'Commercial' ), 'luTypeEx'] = "Commercial"
sdf_ParcelsAA.loc[(sdf_ParcelsAA['luTypeEx'] != "Undevelopable") & ((sdf_ParcelsAA['bldgtype'] != 'Residential') & (sdf_ParcelsAA['bldgtype'] != 'Commercial')), 'luTypeEx'] = "Developable"

In [9]:
##create map centered on Salt Lake
#map_parcels_undevelopable = gis.map('Salt Lake')
#
#countyfilter = 2
#
#
##Filter by County
#sdf_ParcelsAA_lu = sdf_ParcelsAA[sdf_ParcelsAA['county_id']==countyfilter]
##sdf_ParcelsAA_lu
#
##classify land use
#arcade_expression_lu = ("var v = $feature.luTypeEx;"
#                        "if      (v=='Residential'  ) { return 'res'  ; }"
#                        "else if (v=='Commercial'   ) { return 'com'  ; }"
#                        "else if (v=='Developable'  ) { return 'dev'  ; }"
#                        "else if (v=='Undevelopable') { return 'undev'; }"
#                        "else                           { return 'other'  ; }")
#
##symbology for enrollment classes
#uv_lu = [{"value":"res"  , "label":"Residential"  , "symbol":{"type":"esriSFS","color":[255,255,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#         {"value":"com"  , "label":"Commercial"   , "symbol":{"type":"esriSFS","color":[255,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#         {"value":"dev"  , "label":"No Building"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#         {"value":"undev", "label":"Undevelopable", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##define sdf layer
#sdf_ParcelsAA_lu.spatial.plot(map_widget = map_parcels_undevelopable,
#                              renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                              unique_values=uv_lu,
#                              arcade_expression=arcade_expression_lu,
#                              default_symbol="" #don't include an 'other' category
#                              )
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]
#
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_parcels_undevelopable,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_parcels_undevelopable.layout.height='500px'
#map_parcels_undevelopable.legend=True
#
##map title
##display(Markdown('<h2><center>Land Use - County ' + str(countyfilter) + '</center></h2>'))
#display(Markdown('<h2><center>Existing Land Use</center></h2>'))
#
##display map
#map_parcels_undevelopable

# Building Age

In [11]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE'],
      dtype='object')

In [12]:
#set building age with base year 2019
sdf_ParcelsAA['BldAge'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['yearbuilt'] > 0)), 'BldAge'] = 2019 - sdf_ParcelsAA['yearbuilt']
display(sdf_ParcelsAA)
display(sdf_ParcelsAA.columns)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,sfvalue_re,sfvalue_co,value_res,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge
0,0,502.251854,13168.208417,11110,18,a1,1912,579541.0,579541.0,0.0,...,0.000000,0.000000,0.000000,0.000000,3.255373,3.253935e+00,9.995583e-01,3.253935e+00,"{""rings"": [[[407535.2560999999, 4457143.329299...",0.0
1,1,254.142228,4026.737008,11110,18,a1,10577,579551.0,579551.0,1.0,...,82.640882,0.000000,117568.743289,0.000000,0.997945,9.950284e-01,9.970769e-01,9.950284e-01,"{""rings"": [[[407213.32070000004, 4457172.3539]...",43.0
2,2,502.251953,13168.217164,11110,18,a1,31345,579540.0,579540.0,0.0,...,0.000000,0.000000,0.000000,0.000000,3.255376,3.253937e+00,9.995580e-01,3.253937e+00,"{""rings"": [[[407534.25019999966, 4456966.80010...",0.0
3,3,381.212790,8073.508517,11110,18,a1,52886,579564.0,579564.0,0.0,...,0.000000,0.000000,0.000000,0.000000,1.995888,1.995007e+00,9.995589e-01,1.995007e+00,"{""rings"": [[[407360.8212000001, 4457232.3221],...",0.0
4,4,254.141727,4036.751100,11110,18,a1,90204,579574.0,579574.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.997945,9.975029e-01,9.995573e-01,9.975029e-01,"{""rings"": [[[407591.59580000024, 4457228.08950...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939712,939712,0.124294,0.000529,46,5,c3,569725,444878.0,444878.0,0.0,...,0.000000,0.000000,0.000000,0.000000,6.878651,1.307385e-07,1.900642e-08,1.307385e-07,"{""rings"": [[[428818.9419, 4527462.916099999], ...",0.0
939713,939713,0.124294,0.000529,46,5,c3,771763,370287.0,812860.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.050471,1.307385e-07,2.590379e-06,1.307385e-07,"{""rings"": [[[428818.9419, 4527462.916099999], ...",0.0
939714,939714,7.160779,0.311676,47,5,c3,342265,356306.0,356306.0,1.0,...,0.000000,26.459265,0.000000,0.005556,0.401155,7.701683e-05,1.919878e-04,7.701683e-05,"{""rings"": [[[426694.8435000004, 4526659.646299...",30.0
939715,939715,7.160779,0.311676,47,5,c3,359251,408107.0,408107.0,1.0,...,0.000000,61.805978,0.000000,0.008470,0.370131,7.701683e-05,2.080801e-04,7.701683e-05,"{""rings"": [[[426694.8435000004, 4526659.646299...",29.0


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [13]:
##create map centered on Salt Lake
#map_parcels_bldgage = gis.map('Salt Lake')
#
#
#countyfilter = 2
#
##Filter by County
#sdf_ParcelsAA_bldg = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx']!='Undevelopable')]
##sdf_ParcelsAA_lu
#
##expression to classify enrollment
#arcade_expression_bldgage = ("var v = $feature.BldAge;"
#                             "if      (v==0)  { return 'class1'; }"
#                             "else if (v<40)  { return 'class2'; }"
#                             "else if (v<80)  { return 'class3'; }"
#                             "else if (v<120) { return 'class3'; }"
#                             "else            { return 'class4'; }")
#
##symbology for enrollment classes
#uv_public = [{"value":"class1", "label":"No Building"  , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#             {"value":"class2", "label":"Less than 40" , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#             {"value":"class3", "label":"40 to 80"     , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#             {"value":"class4", "label":"80 to 120"    , "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#             {"value":"class4", "label":"More than 120", "symbol":{"type":"esriSFS","color":[  0,  0,  0,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##define sdf layer
#sdf_ParcelsAA_bldg.spatial.plot(map_widget = map_parcels_bldgage,
#                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                unique_values=uv_public,
#                                arcade_expression=arcade_expression_bldgage,
#                                default_symbol="" #don't include an 'other' category
#                                )
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]
#
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_parcels_bldgage,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_parcels_bldgage.layout.height='500px'
#map_parcels_bldgage.legend=True
#
##map title
##display(Markdown('<h2><center>Building Age by Parcel - County ' + str(countyfilter) + '</center></h2>'))
#display(Markdown('<h2><center>Building Age (Years) by Parcel</center></h2>'))
#
##display map
#map_parcels_bldgage

# Parcel Value

In [14]:
sdf_ParcelsAA.columns

Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge'],
      dtype='object')

In [15]:
#set building age with base year 2019
sdf_ParcelsAA['ValuePerAcre'] = 0
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'ValuePerAcre'] = sdf_ParcelsAA['value_res'] / sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'ValuePerAcre'] = sdf_ParcelsAA['value_com'] / sdf_ParcelsAA['Acres']

#classify existing acreage by existing land use

sdf_ParcelsAA['acres_res'] = 0.0
sdf_ParcelsAA['acres_com'] = 0.0

sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Residential')), 'acres_res'] = sdf_ParcelsAA['Acres']
sdf_ParcelsAA.loc[((sdf_ParcelsAA['luTypeEx'] == 'Commercial' )), 'acres_com'] = sdf_ParcelsAA['Acres']

display(sdf_ParcelsAA)

,FID,Shape_Leng,Shape_Area,FID_Analys,AreaID,ClassID,FID_Parcel,parcel_id,parcel_id_,basebldg,...,value_com,ParcelAcre,PieceAcres,PiecePorti,Acres,SHAPE,BldAge,ValuePerAcre,acres_res,acres_com
0,0,502.251854,13168.208417,11110,18,a1,1912,579541.0,579541.0,0.0,...,0.000000,3.255373,3.253935e+00,9.995583e-01,3.253935e+00,"{""rings"": [[[407535.2560999999, 4457143.329299...",0.0,0.000000,0.000000,0.000000
1,1,254.142228,4026.737008,11110,18,a1,10577,579551.0,579551.0,1.0,...,0.000000,0.997945,9.950284e-01,9.970769e-01,9.950284e-01,"{""rings"": [[[407213.32070000004, 4457172.3539]...",43.0,118156.170331,0.995028,0.000000
2,2,502.251953,13168.217164,11110,18,a1,31345,579540.0,579540.0,0.0,...,0.000000,3.255376,3.253937e+00,9.995580e-01,3.253937e+00,"{""rings"": [[[407534.25019999966, 4456966.80010...",0.0,0.000000,0.000000,0.000000
3,3,381.212790,8073.508517,11110,18,a1,52886,579564.0,579564.0,0.0,...,0.000000,1.995888,1.995007e+00,9.995589e-01,1.995007e+00,"{""rings"": [[[407360.8212000001, 4457232.3221],...",0.0,0.000000,0.000000,0.000000
4,4,254.141727,4036.751100,11110,18,a1,90204,579574.0,579574.0,0.0,...,0.000000,0.997945,9.975029e-01,9.995573e-01,9.975029e-01,"{""rings"": [[[407591.59580000024, 4457228.08950...",0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939712,939712,0.124294,0.000529,46,5,c3,569725,444878.0,444878.0,0.0,...,0.000000,6.878651,1.307385e-07,1.900642e-08,1.307385e-07,"{""rings"": [[[428818.9419, 4527462.916099999], ...",0.0,0.000000,0.000000,0.000000
939713,939713,0.124294,0.000529,46,5,c3,771763,370287.0,812860.0,0.0,...,0.000000,0.050471,1.307385e-07,2.590379e-06,1.307385e-07,"{""rings"": [[[428818.9419, 4527462.916099999], ...",0.0,0.000000,0.000000,0.000000
939714,939714,7.160779,0.311676,47,5,c3,342265,356306.0,356306.0,1.0,...,0.005556,0.401155,7.701683e-05,1.919878e-04,7.701683e-05,"{""rings"": [[[426694.8435000004, 4526659.646299...",30.0,72.141558,0.000000,0.000077
939715,939715,7.160779,0.311676,47,5,c3,359251,408107.0,408107.0,1.0,...,0.008470,0.370131,7.701683e-05,2.080801e-04,7.701683e-05,"{""rings"": [[[426694.8435000004, 4526659.646299...",29.0,109.971346,0.000000,0.000077


In [16]:
##create map centered on Salt Lake
#map_parcels_areas_value = gis.map('Salt Lake')
#
#
#
#countyfilter = 2
#
##Filter by County
#sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx'] == 'Residential')]
#
##(sdf_ParcelsAA['luTypeEx']!='Undevelopable') & 
##sdf_ParcelsAA_lu
#
##expression to classify employment growth
##don't use value if 0
#arcade_expression_value = ("var v = $feature.ValuePerAcre;"
#                           "if      (v>0       && v<=1000000) { return 'class4'; }"
#                           "else if (v>1000000 && v<=3000000) { return 'class3'; }"
#                           "else if (v>3000000 && v<=5000000) { return 'class2'; }"
#                           "else if (v>5000000              ) { return 'class1'; }")
#
##symbology for enrollment classes
#uv_value = [{"value":"class1", "label":"More than $5m/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class2", "label":"$5m to $3m/acre"   , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class3", "label":"$3m to $1m/acre"   , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class4", "label":"Less than $1m/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##define sdf layer
#sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
#                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                unique_values=uv_value,
#                                arcade_expression=arcade_expression_value,
#                                default_symbol="" #don't include an 'other' category
#                                )
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]
#
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_parcels_areas_value.layout.height='500px'
#map_parcels_areas_value.legend=True
#
##map title
#display(Markdown('<h2><center>Value per Acre - Residential</center></h2>'))
#
##display map
#map_parcels_areas_value

In [17]:
##create map centered on Salt Lake
#map_parcels_areas_value = gis.map('Salt Lake')
#
#
#
#countyfilter = 2
#
##Filter by County
#sdf_ParcelsAA_value = sdf_ParcelsAA[(sdf_ParcelsAA['county_id']==countyfilter) & (sdf_ParcelsAA['luTypeEx'] == 'Commercial'))]
##sdf_ParcelsAA_lu
#
##expression to classify employment growth
##don't use value if 0
#arcade_expression_value = ("var v = $feature.ValuePerAcre;"
#                           "if      (v>0       && v<=1000000) { return 'class4'; }"
#                           "else if (v>1000000 && v<=3000000) { return 'class3'; }"
#                           "else if (v>3000000 && v<=5000000) { return 'class2'; }"
#                           "else if (v>5000000              ) { return 'class1'; }")
#
##symbology for enrollment classes
#uv_value = [{"value":"class1", "label":"More than 5m/acre" , "symbol":{"type":"esriSFS","color":[130,165,217,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class2", "label":"$3m to $5m/acre"   , "symbol":{"type":"esriSFS","color":[143,111,146,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class3", "label":"$1m to $3m/acre"   , "symbol":{"type":"esriSFS","color":[155, 58, 75,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#            {"value":"class4", "label":"Less than $1m/acre", "symbol":{"type":"esriSFS","color":[168,  4,  4,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##define sdf layer
#sdf_ParcelsAA_value.spatial.plot(map_widget = map_parcels_areas_value,
#                                renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                unique_values=uv_value,
#                                arcade_expression=arcade_expression_value,
#                                default_symbol="" #don't include an 'other' category
#                                )
#
##expression to classify enrollment
#arcade_expression_aa = ("return 'classAA';")
#
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]
#
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_parcels_areas_value,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_parcels_areas_value.layout.height='500px'
#map_parcels_areas_value.legend=True
#
##map title
#display(Markdown('<h2><center>Value per Acre - Commercial</center></h2>'))
#
##display map
#map_parcels_areas_value

# Development Categories

In [18]:
df_ClassParam = pd.read_csv(ClassParameters)
display(df_ClassParam)

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


In [19]:
#ReDevelop, NewDevelop, NoDevelop

sdf_ParcelsAAwParam = pd.DataFrame.merge(sdf_ParcelsAA,df_ClassParam,on='ClassID',how='left')
#display(sdf_ParcelsAAwParam)

#undeveloped
sdf_ParcelsDevCat = sdf_ParcelsAAwParam.copy()
sdf_ParcelsDevCat['DevCategory'] = 'Remain'
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Developable'), 'DevCategory'] = "Developable"

#initialize column
sdf_ParcelsDevCat['DevCategoryAge'] = ''

#check if redevelopable - step 1: building age
#residential
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeRes_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeRes_High']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryAge'] = "Redevelopable"
#non-residential
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['BldAge'] >= sdf_ParcelsDevCat['RedevBldgAgeCom_Low']) & (sdf_ParcelsDevCat['BldAge'] <= sdf_ParcelsDevCat['RedevBldgAgeCom_High']) & (sdf_ParcelsDevCat['luTypeEx'] != 'Residential'), 'DevCategoryAge'] = "Redevelopable"

#initialize column
sdf_ParcelsDevCat['DevCategoryValue'] = ''

#check if redevelopable - step 2: value
#residential
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreRes']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Residential'), 'DevCategoryValue'] = "Redevelopable"
#commercial
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['ValuePerAcre'] > 0) & (sdf_ParcelsDevCat['ValuePerAcre'] <= sdf_ParcelsDevCat['RedevValuePerAcreCom']) & (sdf_ParcelsDevCat['luTypeEx'] == 'Commercial' ), 'DevCategoryValue'] = "Redevelopable"

#depending on AND or OR defenition, determine whether a parcel is eligible for redevelopment
sdf_ParcelsDevCat.loc[ (sdf_ParcelsDevCat['RedevAndOr'] =='AND') &  (sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') & (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable')  , 'DevCategory'] = "Redevelopable"
sdf_ParcelsDevCat.loc[((sdf_ParcelsDevCat['RedevAndOr'] =='OR' ) & ((sdf_ParcelsDevCat['DevCategoryAge'] == 'Redevelopable') | (sdf_ParcelsDevCat['DevCategoryValue'] == 'Redevelopable'))), 'DevCategory'] = "Redevelopable"

#undevelopable parcels
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['luTypeEx'] == 'Undevelopable'), 'DevCategory'] = "Undevelopable"


#single-family redevelopment additional criteria (redeveloped parcel must include enough area for at least 1.0 more unit than original)

#calculate redev potential for SF units... resunits range from 0.95 to 1.05 to account for rounding error
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['DevCategory'] == 'Redevelopable') & (sdf_ParcelsDevCat['SFRedevFullAdd'] =='Yes') & (sdf_ParcelsDevCat['resunits'] > 0.95) & (sdf_ParcelsDevCat['resunits'] < 1.05), 'SFRedevAdd'] = sdf_ParcelsDevCat['CapacityRes_DUA'] * (sdf_ParcelsDevCat['Acres'] * (1-sdf_ParcelsDevCat['PercentOpenSpace'])) - sdf_ParcelsDevCat['resunits']

#change to 'Remain' for SF parcels with not enough redev space for additional units
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['DevCategory'] == 'Redevelopable') & (sdf_ParcelsDevCat['SFRedevFullAdd'] =='Yes') & (sdf_ParcelsDevCat['resunits'] > 0.95) & (sdf_ParcelsDevCat['resunits'] < 1.05) & (sdf_ParcelsDevCat['SFRedevAdd'] < 1.0), 'DevCategory']      = 'Remain'

#changed to Remain
sdf_ParcelsDevCat.loc[(sdf_ParcelsDevCat['SFRedevAdd'] < 1.0), 'DevCategory_Flag'] = 'SFRedevAdd_NoPotential'

#export to GeoDB
display("exporting to gdb...")
deleteIfExists(ParcelsDevCategory)
sdf_ParcelsDevCat.spatial.to_featureclass(location=ParcelsDevCategory)

display(sdf_ParcelsDevCat[['yearbuilt','luTypeEx','ValuePerAcre','BldAge','DevCategory']])

sdf_ParcelsDevCat.columns

'exporting to gdb...'

,yearbuilt,luTypeEx,ValuePerAcre,BldAge,DevCategory
0,0.0,Developable,0.000000,0.0,Developable
1,1976.0,Residential,118156.170331,43.0,Remain
2,0.0,Developable,0.000000,0.0,Developable
3,0.0,Developable,0.000000,0.0,Developable
4,0.0,Developable,0.000000,0.0,Developable
...,...,...,...,...,...
939712,0.0,Developable,0.000000,0.0,Developable
939713,0.0,Developable,0.000000,0.0,Developable
939714,1989.0,Commercial,72.141558,30.0,Redevelopable
939715,1990.0,Commercial,109.971346,29.0,Redevelopable


Index(['FID', 'Shape_Leng', 'Shape_Area', 'FID_Analys', 'AreaID', 'ClassID',
       'FID_Parcel', 'parcel_id', 'parcel_id_', 'basebldg', 'parcel_sqf',
       'parcel_acr', 'county_id', 'max_dua', 'max_far', 'max_height',
       'luTypeEx', 'max_dua_ce', 'max_far_ce', 'bldgtype', 'yearbuilt',
       'resunits', 'job_spaces', 'sf_res', 'sf_com', 'sfvalue_re',
       'sfvalue_co', 'value_res', 'value_com', 'ParcelAcre', 'PieceAcres',
       'PiecePorti', 'Acres', 'SHAPE', 'BldAge', 'ValuePerAcre', 'acres_res',
       'acres_com', 'ClassDescription', 'SFSplitRes', 'SFSplitCom',
       'CapacityRes_DUA', 'CapacityCom_FAR', 'SFperHH', 'SFperEmp',
       'PercentOpenSpace', 'RedevValuePerAcreRes', 'RedevValuePerAcreCom',
       'RedevAndOr', 'RedevBldgAgeRes_Low', 'RedevBldgAgeRes_High',
       'RedevBldgAgeCom_Low', 'RedevBldgAgeCom_High', 'RedevProb',
       'SFRedevFullAdd', 'ClassOrder', 'DevCategory', 'DevCategoryAge',
       'DevCategoryValue', 'SFRedevAdd', 'DevCategory_Flag'],
      d

In [20]:
#sdf_ParcelsWebApp = sdf_ParcelsDevCat[['county_id','AreaID','ClassID','parcel_id','luTypeEx','BldAge','ValuePerAcre','DevCategory','PieceAcres','job_spaces','resunits','SHAPE']]
##sdf_ParcelsWebApp
#
##if processing geodatabase doesn't exist, create it
#print("Checking if " + WebAppGDB + " exists...")
#if not arcpy.Exists(dir_webapp):
#    print("Creating " + WebAppGDB + "...")
#    arcpy.management.CreateFileGDB(dir_webapp, WebAppGDB)
#else:
#    print(WebAppGDB + " exists...")
#
##export to GeoDB
#display("exporting to gdb...")
#deleteIfExists(ParcelsWebApp)
#sdf_ParcelsWebApp.spatial.to_featureclass(location=ParcelsWebApp)
#display("repairing geometry...")
#arcpy.RepairGeometry_management(ParcelsWebApp)

In [21]:
##dissolve on DevCategory
#deleteIfExists(DevCatWebApp)
#arcpy.management.Dissolve(ParcelsWebApp, DevCatWebApp, dissolve_field=['AreaID','ClassID','DevCategory'], statistics_fields=[['PieceAcres','SUM'],['job_spaces','SUM'],['resunits','SUM']])
#
#repair the layer
#arcpy.RepairGeometry_management(DevCatWebApp)

In [22]:
display(df_ClassParam)
df_ClassParam.dtypes

,ClassID,ClassDescription,SFSplitRes,SFSplitCom,CapacityRes_DUA,CapacityCom_FAR,SFperHH,SFperEmp,PercentOpenSpace,RedevValuePerAcreRes,RedevValuePerAcreCom,RedevAndOr,RedevBldgAgeRes_Low,RedevBldgAgeRes_High,RedevBldgAgeCom_Low,RedevBldgAgeCom_High,RedevProb,SFRedevFullAdd,ClassOrder
0,f4,Multifamily Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,29
1,f3,Multifamily City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,28
2,f2,Multifamily Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,27
3,f1,Multifamily Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,26
4,g4,Mixed-Use Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,33
5,g3,Mixed-Use City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,32
6,g2,Mixed-Use Suburban,0.70,0.30,30.0,1.0,1500,300,0.10,1500000,1200000,AND,60,120,20,120,1,No,31
7,g1,Mixed-Use Single Story,0.85,0.15,10.0,0.5,2000,300,0.10,1000000,800000,AND,60,300,20,300,1,No,30
8,h4,Mixed Residential Metro,0.40,0.60,120.0,15.0,1500,225,0.10,5000000,4000000,AND,60,120,20,120,1,No,37
9,h3,Mixed Residential City/TOD,0.50,0.50,60.0,4.0,1500,250,0.10,3000000,2400000,AND,60,120,20,120,1,No,36


ClassID                  object
ClassDescription         object
SFSplitRes              float64
SFSplitCom              float64
CapacityRes_DUA         float64
CapacityCom_FAR         float64
SFperHH                   int64
SFperEmp                  int64
PercentOpenSpace        float64
RedevValuePerAcreRes      int64
RedevValuePerAcreCom      int64
RedevAndOr               object
RedevBldgAgeRes_Low       int64
RedevBldgAgeRes_High      int64
RedevBldgAgeCom_Low       int64
RedevBldgAgeCom_High      int64
RedevProb                 int64
SFRedevFullAdd           object
ClassOrder                int64
dtype: object

In [23]:
#countyfilter = 2
#
##Filter by County
#sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat.copy()
#sdf_ParcelsDevCat_filter = sdf_ParcelsDevCat_filter[sdf_ParcelsDevCat_filter['county_id']==countyfilter]
#
##create map centered on Salt Lake
#map_parcels_dev = gis.map('Salt Lake')
#
##expression to classify enrollment
#arcade_expression_dev = ("var v = $feature.DevCategory;"
#                             "if      (v=='Undevelopable') { return 'class2'; }"
#                             "else if (v=='Remain'       ) { return 'class2'; }"
#                             "else if (v=='Redevelopable') { return 'class3'; }"
#                             "else if (v=='Developable'  ) { return 'class4'; }"
#                        )
#
##symbology for enrollment classes
#uv_dev = [{"value":"class4", "label":"Developable"  , "symbol":{"type":"esriSFS","color":[  0,  0,255,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"class3", "label":"Redevelopable", "symbol":{"type":"esriSFS","color":[128,  0,128,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}},
#          {"value":"class2", "label":"To Remain"    , "symbol":{"type":"esriSFS","color":[180,180,180,168], "outline":{"type":"esriSLS","color":[255,255,255,51], "width":1.5,"style":"esriSLSSolid"}, "style":"esriSFSSolid"}}]
#
##define sdf layer
#sdf_ParcelsDevCat_filter.spatial.plot(map_widget = map_parcels_dev,
#                                      renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                      unique_values=uv_dev,
#                                      arcade_expression=arcade_expression_dev,
#                                      default_symbol="" #don't include an 'other' category
#                                      )
##symbology for enrollment classes
#uv_aa = [{"value":"classAA", "label":"Analysis Area", "symbol":{"type":"esriSFS","color":[0,0,0,128],"outline":{"type":"esriSLS","color":[0,0,0,128], "width":2,"style":"esriSLSSolid"}, "style":"esriSFSNull"}}]
#
#
##define sdf layer
#sdf_AA.spatial.plot(map_widget = map_parcels_dev,
#                                 renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
#                                 unique_values=uv_aa,
#                                 arcade_expression=arcade_expression_aa,
#                                 default_symbol="" #don't include an 'other' category
#                                 )
#
##define map characteristics
#map_parcels_dev.layout.height='500px'
#map_parcels_dev.legend=True
#
##map title
#display(Markdown('<h2><center>Development Category - County ' + str(countyfilter) + '</center></h2>'))
#
##display map
#map_parcels_dev

In [24]:
#aggregate parcels into analysis areas / classes
df_AADevCat = sdf_ParcelsDevCat.groupby(['AreaID','ClassID','county_id','DevCategory'],as_index=False).agg({"parcel_id": [np.size], "job_spaces": [np.sum], "resunits":[np.sum], "Acres":[np.sum], "sf_res":[np.sum], "sf_com":[np.sum], "acres_res":[np.sum], "acres_com":[np.sum]})
df_AADevCat.columns = df_AADevCat.columns.droplevel(1)
display(df_AADevCat)


df_AADevCat.to_csv(os.path.join(dir_results, r'AnalysisAreas_byDevCategory.csv'),index=False)


,AreaID,ClassID,county_id,DevCategory,parcel_id,job_spaces,resunits,Acres,sf_res,sf_com,acres_res,acres_com
0,0,a1,0.0,Developable,18.0,0.000000e+00,0.000000,15.567788,0.000000,0.000000,0.000000,0.000000
1,0,a1,0.0,Undevelopable,8.0,0.000000e+00,0.000000,14.733847,0.000000,0.000000,0.000000,0.000000
2,0,a1,1.0,Undevelopable,1.0,0.000000e+00,0.000000,0.108287,0.000000,0.000000,0.000000,0.000000
3,0,a1,2.0,Developable,102.0,0.000000e+00,0.000000,234.281316,0.000000,0.000000,0.000000,0.000000
4,0,a1,2.0,Redevelopable,10.0,4.323625e-02,8.002301,84.596637,10868.498573,42.155347,83.663641,0.932996
...,...,...,...,...,...,...,...,...,...,...,...,...
2446,77,f2,1.0,Undevelopable,550.0,5.958049e+00,0.000000,15.171574,0.000000,2228.310208,0.000000,0.000000
2447,77,g2,1.0,Developable,39.0,0.000000e+00,0.000000,94.609768,0.000000,0.000000,0.000000,0.000000
2448,77,g2,1.0,Redevelopable,2.0,5.942816e-09,0.998350,0.320516,1186.039267,0.000002,0.320457,0.000059
2449,77,g2,1.0,Remain,14.0,5.365716e-06,5.961774,5.687062,15730.287215,0.003396,5.682825,0.004237
